In [ ]:
import pickle
import os
import math

filenames = ['chance1.0',
             'chance0.9', 'chance0.9_2', 'chance0.9_3', 'chance0.9_4', 'chance0.9_5',
             'chance0.8_run1', 'chance0.8_run2', 'chance0.8_run3', 'chance0.8_run4', 'chance0.8_run5',
             'chance0.7_run1', 'chance0.7_run2', 'chance0.7_run3', 'chance0.7_run4', 'chance0.7_run5',
             '0.6 first run', '0.6 second run', '0.6 third run', '0.6 fourth run', '0.6 fifth run',
             'chance0.5_run1', 'chance0.5_run2', 'chance0.5_run3', 'chance0.5_run4', 'chance0.5_run5']

chances = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5]
hist_data = dict()

for i, filename in enumerate(filenames):
    with open(os.path.join(os.getcwd(), f'runs\\{filename}.pkl'), 'rb') as file:
        chance = chances[math.floor((i + 4) / 5)]
        if not chance in hist_data:
            hist_data[chance] = []
        hist_data[chance].append(pickle.load(file))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

overcorrection_rate = 1.0 - np.array(chances)

for bin in range(2):
    hist_counts = {}
    for chance in chances:
        # The average of the 5 runs is computed with the mean.
        # Afterwards, only the data of the currently chosen bin is taken (column).
        hist_counts[chance] = np.array(hist_data[chance]).mean(axis = 0)[:, bin]
        
    hist_counts_df = pd.DataFrame(hist_counts)
    corr_matrix = hist_counts_df.corr(method='kendall')

    fig = plt.figure(figsize=(9, 8))
    plt.plot(overcorrection_rate, corr_matrix[1.0], color="green")
    plt.title(
        f"""Kendall Tau correlation with 0$\%$ 
        overcorrection case for {bin*5 + 5} km proximity""", fontsize=30)
    plt.ylabel("Correlation (-1 to 1)", fontsize=25)
    plt.xlabel("Chance of overcorrection", fontsize=25)
    plt.yticks(fontsize=18)
    plt.xticks(fontsize=18)

    plt.savefig(f"kendall_corr_{bin*5 + 5}km.png")
    plt.show()


    hist_seconds = hist_counts_df.sum(axis = 0).values

    fig = plt.subplots(figsize=(9, 8))
    plt.plot(overcorrection_rate, hist_seconds)
    plt.ylabel("Seconds", fontsize=25)
    plt.yticks(fontsize=18)
    plt.xticks(fontsize=18)
    plt.title(
        f"""Amount of seconds that satellites spend within 
        a proximity of {bin*5 + 5} km of eachother vs chance of 
        overcorrection by starlink boosters""",
        fontsize=30,
    )
    plt.xlabel("Chance of overcorrection", fontsize=25)
    plt.savefig(f"proximity_analysis_{bin*5 + 5}km.png")
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

hist_counts_0_5 = np.array(hist_data[0.5]).mean(axis = 0)
hist_counts_0_6 = np.array(hist_data[0.6]).mean(axis = 0)
hist_counts_0_7 = np.array(hist_data[0.7]).mean(axis = 0)
hist_counts_0_8 = np.array(hist_data[0.8]).mean(axis = 0)
hist_counts_0_9 = np.array(hist_data[0.9]).mean(axis = 0)
hist_counts_1_0 = np.array(hist_data[1.0]).mean(axis = 0)

data = {'0.5':  hist_counts_0_5[:, 1],
        '0.6':  hist_counts_0_6[:, 1],
        '0.7':  hist_counts_0_7[:, 1],
        '0.8':  hist_counts_0_8[:, 1],
        '0.9':  hist_counts_0_9[:, 1],
        '1.0':  hist_counts_1_0[:, 1]
        }
data2 = {'Total counts':  [np.sum(hist_counts_0_5[:, 1]), np.sum(hist_counts_0_6[:, 1]), np.sum(hist_counts_0_7[:, 1]), np.sum(hist_counts_0_8[:, 1]), np.sum(hist_counts_0_9[:, 1]), np.sum(hist_counts_1_0[:, 1])],
        'Chances':  [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        }

print(data)

df = pd.DataFrame(data)
df2 = pd.DataFrame(data2)
print(f"{df2}\n")
corr_matrix = df.corr(method='kendall')
print(corr_matrix)
df2["Chances"] = 1 - df2["Chances"] 

fig = plt.figure(figsize=(9, 8))
plt.plot(df2["Chances"], df2["Total counts"], color="blue")
plt.ylabel("Seconds", fontsize=25)
plt.yticks(fontsize=18)
plt.xticks(fontsize=18)
plt.title(
    f"""Amount of seconds that satellites spend within 
    a proximity of 10 km of eachother vs chance of 
    overcorrection by starlink boosters""",
    fontsize=30,
)
plt.xlabel("Chance of overcorrection", fontsize=25)
plt.savefig("proximity_analysis_10km.png")
plt.show()

fig = plt.figure(figsize=(9, 8))
plt.plot(df2["Chances"], corr_matrix['1.0'], color="green")
plt.title(
    """Kendall Tau correlation with 0$\%$ 
    overcorrection case for 10 km proximity""", fontsize=30)
plt.ylabel("Correlation (-1 to 1)", fontsize=25)
plt.xlabel("Chance of overcorrection", fontsize=25)
plt.yticks(fontsize=18)
plt.xticks(fontsize=18)

plt.savefig("kendall_corr_10km.png")
plt.show()